In [13]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from glob import glob
import numpy as np
import os,re,string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")

## IMDB dataset and the sentiment classification task

In [3]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-06-26 01:03:05--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz     0%[                    ] 121.30K   172KB/s               ^C


In [4]:
!gunzip aclImdb_v1.tar.gz

aclImdb_v1.tar already exists -- do you wish to overwrite (y or n)? ^C


In [124]:
!tar -xvf aclImdb_v1.tar

### Tokenizing and term document matrix creation

In [14]:
PATH='data/aclImdb/'
names = ['neg','pos']

In [15]:
!ls {PATH}

README     imdb.vocab imdbEr.txt test       train


In [16]:
!ls {PATH}train

labeledBow.feat pos             unsupBow.feat   urls_pos.txt
neg             unsup           urls_neg.txt    urls_unsup.txt


In [17]:
!ls {PATH}train/pos | head

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt


In [18]:
!ls {PATH}test

labeledBow.feat neg             pos             urls_neg.txt    urls_pos.txt


In [19]:
!ls {PATH}test/pos | head

0_10.txt
10000_7.txt
10001_9.txt
10002_8.txt
10003_8.txt
10004_9.txt
10005_8.txt
10006_7.txt
10007_10.txt
10008_8.txt


In [20]:
def load_texts_labels_from_folders(path, folders):
    texts,labels = [],[]
    for idx,label in enumerate(folders):
        for fname in glob(os.path.join(path, label, '*.*')):
            texts.append(open(fname, 'r').read())
            labels.append(idx)
    # stored as np.int8 to save space 
    return texts, np.array(labels).astype(np.int8)

In [21]:
trn,trn_y = load_texts_labels_from_folders(f'{PATH}train',names)
val,val_y = load_texts_labels_from_folders(f'{PATH}test',names)

In [22]:
len(val),len(trn_y),len(val),len(val_y)

(25000, 25000, 25000, 25000)

In [23]:
len(trn_y[trn_y==1]),len(val_y[val_y==1])

(12500, 12500)

In [24]:
type(trn_y),type(trn)

(numpy.ndarray, list)

In [25]:
np.unique(trn_y)

array([0, 1], dtype=int8)

In [26]:
??texts_labels_from_folders

Object `texts_labels_from_folders` not found.


Here is the text of the first review

In [27]:
print(trn[0])
print()
print(f"Review's label: {trn_y[0]}")
# 0 represent a negative review

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.

Review's label: 0


In [28]:
trn_y[0] # 0 represent a negative review

0

[`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) converts a collection of text documents to a matrix of token counts (part of `sklearn.feature_extraction.text`).

In [29]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [30]:
s = "This 'movie' isn’t good." 
tokenize(s)

['This', "'", 'movie', "'", 'isn', '’', 't', 'good', '.']

In [31]:
#create term documetn matrix
veczr = CountVectorizer(tokenizer=tokenize)

`fit_transform(trn)` finds the vocabulary in the training set. It also transforms the training set into a term-document matrix. Since we have to apply the *same transformation* to your validation set, the second line uses just the method `transform(val)`. `trn_term_doc` and `val_term_doc` are sparse matrices. `trn_term_doc[i]` represents training document i and it contains a count of words for each document for each word in the vocabulary.

In [32]:
trn_term_doc = veczr.fit_transform(trn)
# Important: Use same vocab for validation set
val_term_doc = veczr.transform(val)

In [33]:
trn_term_doc

<25000x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 3749745 stored elements in Compressed Sparse Row format>

In [34]:
trn_term_doc[5] #83 stored elements

<1x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 83 stored elements in Compressed Sparse Row format>

In [35]:
w0 = set([o.lower() for o in trn[5].split(' ')]); w0

{'"it',
 '"ripping',
 'a',
 'accusations',
 'agree',
 'allen',
 "allen's",
 'and',
 'appears',
 'applauded',
 'are',
 'bergman',
 'bergman,',
 'bergman.',
 'brian',
 'but',
 'contemporaneous',
 'contrary',
 'critics',
 'cultural',
 'depalma',
 "don't",
 'drama',
 'excoriated',
 'films?',
 'find',
 'for',
 'form',
 'generally',
 'get',
 'good',
 'have',
 'his',
 'hitchcock',
 'i',
 'idea',
 'imitating',
 'imitations',
 'in',
 'is',
 "it's",
 'kid',
 "let's",
 'many',
 'mostly',
 'not',
 'notwithstanding.',
 'of',
 'off"',
 'originality',
 'ourselves:',
 'pretensions,',
 'pretentious',
 'reason:',
 'robin',
 'snobbery.',
 'strange',
 'supportive',
 'suspense/horror',
 'that',
 'that.',
 'the',
 'they',
 'this:',
 'to',
 'unbearably',
 'unpalatable."',
 'view,',
 'was',
 'were',
 'what',
 'whining',
 'why',
 'with',
 "wood's",
 'wooden',
 'woody',
 'would'}

In [36]:
len(w0)
# length is 78 which is pretty similar to 83, and just the 
# difference will be that I didn’t use a real tokenizer. 

78

In [37]:
vocab = veczr.get_feature_names()
print(len(vocab))
vocab[5000:5005]

75132


['aussie', 'aussies', 'austen', 'austeniana', 'austens']

In [38]:
veczr.vocabulary_['absurd']

1297

In [39]:
trn_term_doc[0,1297]
# word 'absurb' appears twice in the first document

2

In [40]:
vocab[4050]

'arching'

In [41]:
veczr.vocabulary_['arching']

4050

In [42]:
trn_term_doc[0,4050]
# word 'arching' does not appear in the first document

0

## Naive Bayes

In [43]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [44]:
x=trn_term_doc
y=trn_y

r = np.log((pr(1)/pr(0)))
b = np.log((y==1).mean() / (y==0).mean())

In [45]:
r.shape,val_term_doc.shape

((1, 75132), (25000, 75132))

Here is the formula for Naive Bayes.

In [46]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.81656

...and binarized Naive Bayes.

In [47]:
x=trn_term_doc.sign()
r = np.log(pr(1)/pr(0))

pre_preds = val_term_doc.sign() @ r.T + b #sign binarize
preds = pre_preds.T>0
(preds==val_y).mean()

0.83016

### Logistic Regression

In [48]:
trn_term_doc.shape

(25000, 75132)

In [49]:
x=trn_term_doc
y=trn_y
m = LogisticRegression(C=1e8, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.85248

...and binarized Logistic Regression.

In [50]:
m = LogisticRegression(C=1e8, dual=True) #low C big regularization
m.fit(trn_term_doc.sign(), y) #present or abscent of words
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.85512

...and the regularized version. You would think 75,000 parameters for 25,000 documents, it’s likely to overfit. Indeed, it did overfit. So this is adding L2 regularization to avoid overfitting (increase C).

In [51]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.88292

In [52]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.88404

### Trigram with NB features

Our next model is a version of logistic regression with Naive Bayes features described [here](https://www.aclweb.org/anthology/P12-2018). For every document we compute binarized features as described above, but this time we use bigrams and trigrams too. Each feature is a log-count ratio. A logistic regression model is then trained to predict sentiment.

In [53]:
veczr =  CountVectorizer(ngram_range=(1,3), 
                         tokenizer=tokenize, max_features=800000)
# limit featutes to 800,000
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [54]:
trn_term_doc.shape

(25000, 800000)

In [55]:
# build voc
vocab = veczr.get_feature_names()

In [56]:
vocab[300000:300010]

['for what seemed',
 'for what seems',
 'for what she',
 'for what the',
 'for what they',
 'for what was',
 'for what we',
 'for what you',
 'for whatever',
 'for whatever reason']

In [57]:
y=trn_y
x=trn_term_doc
m = LogisticRegression(C=0.1, dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.90196

...and binarized Logistic Regression.

In [58]:
m = LogisticRegression(C=0.1, dual=True)
m.fit(x.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.905

In [59]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [60]:
r = np.log(pr(1) / pr(0))
b = np.log((y==1).mean() / (y==0).mean())

Here is the $\text{log-count ratio}$ `r`.  

In [61]:
r.shape, r

((1, 800000), matrix([[-7.51106117e-02, -3.38581347e-04,  1.05483434e-01, ...,
           1.38629436e+00, -2.07944154e+00, -2.07944154e+00]]))

In [62]:
np.exp(r)

matrix([[0.92764087, 0.99966148, 1.1112477 , ..., 4.        , 0.125     ,
         0.125     ]])

Here we fit regularized logistic regression where the features are the trigrams' log-count ratios.

In [63]:
x_nb = x.multiply(r)
m = LogisticRegression(dual=True, C=0.1)
m.fit(x_nb, y);

val_x_nb = val_term_doc.multiply(r)
preds = m.predict(val_x_nb)
(preds.T==val_y).mean()

0.91556

In [69]:
x_nb = x.multiply(r)
m = LogisticRegression(dual=True, C=1e-8)
m.fit(x_nb, y);

val_x_nb = val_term_doc.multiply(r)
preds = m.predict(val_x_nb)
(preds.T==val_y).mean()

0.78024